## Import Libraries

In [1]:
import pandas as pd
import time
import numpy as np
import math
from itertools import islice 
import json
from pandas.io.json import json_normalize
import os

## Read and parse json files for each video

In [47]:


# this finds our json files
path_to_json = "/mnt/sde/jagadish/userdata/dl_project/hoc_json_files/json_files_new/jag_8379/"
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

# here I define my pandas Dataframe with the columns I want to get from the json
jsons_data = pd.DataFrame(columns=['index','person_id','po'])
df = pd.DataFrame()


# we need both the json and an index number so use enumerate()
for index, js in enumerate(json_files):
    with open(os.path.join(path_to_json, js)) as json_file:
        json_text = json.load(json_file)
        index = (str(json_file).split('/')[-1].split('_')[2])
        person_id = json_text['people'][0]['person_id']
        po = json_text['people'][0]['pose_keypoints_2d']
        # here I push a list of data into a pandas DataFrame at row given by 'index'
        jsons_data.loc[index] = [index,person_id, po]


# now that we have the pertinent json data in our DataFrame let's look at it
print(jsons_data)

                     index person_id  \
000000000171  000000000171      [-1]   
000000000123  000000000123      [-1]   
000000000254  000000000254      [-1]   
000000000855  000000000855      [-1]   
000000000422  000000000422      [-1]   
000000000454  000000000454      [-1]   
000000000464  000000000464      [-1]   
000000000722  000000000722      [-1]   
000000000544  000000000544      [-1]   
000000000669  000000000669      [-1]   
000000000354  000000000354      [-1]   
000000000303  000000000303      [-1]   
000000000017  000000000017      [-1]   
000000000016  000000000016      [-1]   
000000000842  000000000842      [-1]   
000000000638  000000000638      [-1]   
000000000619  000000000619      [-1]   
000000000847  000000000847      [-1]   
000000000086  000000000086      [-1]   
000000000218  000000000218      [-1]   
000000000317  000000000317      [-1]   
000000000002  000000000002      [-1]   
000000000688  000000000688      [-1]   
000000000533  000000000533      [-1]   


## Preprocessing data

In [48]:
df = jsons_data

In [49]:
df['index'] = df['index'].astype(int)

In [50]:
df=df.sort_index()

In [51]:
df.head(5)

,index,person_id,po
000000000000,0,[-1],"[268.211, 456.696, 0.855037, 433.031, 447.888,..."
000000000001,1,[-1],"[268.25, 456.659, 0.857249, 433.096, 447.823, ..."
000000000002,2,[-1],"[268.286, 456.711, 0.874095, 433.096, 444.981,..."
000000000003,3,[-1],"[268.295, 456.743, 0.865399, 433.097, 444.936,..."
000000000004,4,[-1],"[268.326, 456.739, 0.861132, 433.12, 444.938, ..."


In [52]:
gf = pd.DataFrame()

In [53]:
gf=df

In [54]:
gf.head(1)

,index,person_id,po
000000000000,0,[-1],"[268.211, 456.696, 0.855037, 433.031, 447.888,..."


In [55]:
gf = gf.join(gf['po'].apply(pd.Series).add_prefix('po'))

In [56]:
gf = gf.drop(['po','person_id','index'], axis=1)

In [57]:
gf.head(1)

,po0,po1,po2,po3,po4,po5,po6,po7,po8,po9,...,po65,po66,po67,po68,po69,po70,po71,po72,po73,po74
000000000000,268.211,456.696,0.855037,433.031,447.888,0.667775,436.098,589.15,0.694529,583.232,...,0.114276,1560.42,612.679,0.363154,1545.8,612.712,0.342137,1492.75,550.922,0.244183


In [58]:
gf = gf.drop(gf.iloc[:, 2::3],axis=1)

In [59]:
gf.head(1)

,po0,po1,po3,po4,po6,po7,po9,po10,po12,po13,...,po60,po61,po63,po64,po66,po67,po69,po70,po72,po73
000000000000,268.211,456.696,433.031,447.888,436.098,589.15,583.232,736.263,527.34,603.847,...,0.0,0.0,1380.88,430.1,1560.42,612.679,1545.8,612.712,1492.75,550.922


In [60]:
mf = pd.DataFrame()

In [61]:
n = gf.shape[1] 
i=0
j=0
while i < n:

    col = gf.columns[i][0:2] + "_" + str(j) #col name
    X = gf.columns[i]
    Y = gf.columns[i+1]
    mf[col] = gf[[X, Y]].values.tolist()
    i = i+2
    j= j+1

In [62]:
mf.head(1)

,po_0,po_1,po_2,po_3,po_4,po_5,po_6,po_7,po_8,po_9,...,po_15,po_16,po_17,po_18,po_19,po_20,po_21,po_22,po_23,po_24
0,"[268.211, 456.696]","[433.031, 447.888]","[436.098, 589.15]","[583.232, 736.263]","[527.34, 603.847]","[433.12, 335.976]","[542.02, 168.246]","[524.411, 288.842]","[927.599, 456.703]","[918.81, 527.314]",...,"[247.652, 486.09]","[244.847, 430.195]","[268.33, 527.285]","[271.191, 386.043]","[0.0, 0.0]","[0.0, 0.0]","[1380.88, 430.1]","[1560.42, 612.679]","[1545.8, 612.712]","[1492.75, 550.922]"


In [63]:
hf=pd.DataFrame()

In [64]:
hf = mf

In [65]:
hf.head(1)

,po_0,po_1,po_2,po_3,po_4,po_5,po_6,po_7,po_8,po_9,...,po_15,po_16,po_17,po_18,po_19,po_20,po_21,po_22,po_23,po_24
0,"[268.211, 456.696]","[433.031, 447.888]","[436.098, 589.15]","[583.232, 736.263]","[527.34, 603.847]","[433.12, 335.976]","[542.02, 168.246]","[524.411, 288.842]","[927.599, 456.703]","[918.81, 527.314]",...,"[247.652, 486.09]","[244.847, 430.195]","[268.33, 527.285]","[271.191, 386.043]","[0.0, 0.0]","[0.0, 0.0]","[1380.88, 430.1]","[1560.42, 612.679]","[1545.8, 612.712]","[1492.75, 550.922]"


In [66]:
hf.shape[1]

25

## Euclidean distances between body key points

In [67]:
def eudis5(v1, v2): # Function to calculate euclidean distance between two points
    dist = [(a - b)**2 for a, b in zip(v1, v2)]
    dist = math.sqrt(sum(dist))
    return dist

In [68]:
hf['pd'] = ''
hf['pd'] = hf['pd'].apply(list)

In [69]:
n=hf.index
m = hf.shape[1]
for i in n[:] :
    
    ear =[]
    I=1
    for j in range(25) :
        
        for k in range(I,25) :

            X = hf.columns[j]
            Y = hf.columns[k]

            a = np.array(hf[X][i])
            b = np.array(hf[Y][i])
            x = eudis5(a, b)
            ear.append(x)
        I = I + 1
     
    hf.loc[i,'pd'].append(ear[:])

In [70]:
hf.head()

,po_0,po_1,po_2,po_3,po_4,po_5,po_6,po_7,po_8,po_9,...,po_16,po_17,po_18,po_19,po_20,po_21,po_22,po_23,po_24,pd
0,"[268.211, 456.696]","[433.031, 447.888]","[436.098, 589.15]","[583.232, 736.263]","[527.34, 603.847]","[433.12, 335.976]","[542.02, 168.246]","[524.411, 288.842]","[927.599, 456.703]","[918.81, 527.314]",...,"[244.847, 430.195]","[268.33, 527.285]","[271.191, 386.043]","[0.0, 0.0]","[0.0, 0.0]","[1380.88, 430.1]","[1560.42, 612.679]","[1545.8, 612.712]","[1492.75, 550.922]","[[165.05518248149616, 213.84598870448795, 421...."
1,"[268.25, 456.659]","[433.096, 447.823]","[436.106, 589.115]","[583.275, 736.288]","[527.359, 603.882]","[433.163, 333.091]","[539.158, 168.289]","[524.404, 286.001]","[927.644, 459.646]","[927.564, 530.267]",...,"[244.904, 430.209]","[268.352, 527.332]","[271.216, 386.047]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[1563.31, 609.794]","[1551.62, 612.724]","[1501.59, 550.797]","[[165.08264176466284, 213.8228908980514, 421.2..."
2,"[268.286, 456.711]","[433.096, 444.981]","[436.067, 586.237]","[583.267, 736.277]","[527.377, 603.906]","[433.08, 336.003]","[542.061, 171.122]","[524.37, 286.044]","[927.645, 456.737]","[924.648, 527.362]",...,"[247.628, 430.192]","[268.233, 527.277]","[271.322, 386.012]","[1542.86, 321.404]","[1542.81, 330.114]","[1460.36, 418.435]","[1563.3, 612.705]","[1551.57, 615.6]","[1504.48, 550.844]","[[165.22690156266927, 211.96096017191465, 421...."
3,"[268.295, 456.743]","[433.097, 444.936]","[436.05, 583.313]","[583.292, 736.209]","[527.363, 603.905]","[433.193, 333.09]","[539.132, 171.181]","[524.415, 288.862]","[927.568, 456.725]","[918.763, 527.33]",...,"[244.912, 430.245]","[268.238, 527.274]","[271.284, 386.031]","[1545.68, 321.423]","[1545.67, 330.187]","[1451.54, 421.373]","[1563.32, 612.691]","[1551.62, 615.583]","[1501.58, 550.851]","[[165.2244063478516, 210.1468651324592, 421.09..."
4,"[268.326, 456.739]","[433.12, 444.938]","[436.038, 583.297]","[586.172, 733.39]","[527.355, 603.877]","[433.127, 336.051]","[544.94, 171.215]","[524.443, 288.888]","[930.581, 456.692]","[927.599, 527.358]",...,"[247.661, 433.023]","[268.37, 527.302]","[271.211, 386.106]","[0.0, 0.0]","[0.0, 0.0]","[1413.28, 430.143]","[1563.36, 609.794]","[1551.58, 612.704]","[1501.54, 550.818]","[[165.21599812669473, 210.10531242212798, 421...."


## Angle between upper_arm and fore_arm_left_hand

In [71]:
n=hf.index
for i in n[:] :
    


    X = 'po_5'
    Y = 'po_6'
    Z = 'po_7'

    a = np.array(hf[X][i])
    b = np.array(hf[Y][i])
    c = np.array(hf[Z][i])
    
    ba = a - b
    bc = c - b
    

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    
    if np.isnan(angle):
        hf.loc[i,'hl_angle'] = 0
    else:
        hf.loc[i,'hl_angle'] = np.degrees(angle)
    

    

  

In [72]:
hf.head()

,po_0,po_1,po_2,po_3,po_4,po_5,po_6,po_7,po_8,po_9,...,po_17,po_18,po_19,po_20,po_21,po_22,po_23,po_24,pd,hl_angle
0,"[268.211, 456.696]","[433.031, 447.888]","[436.098, 589.15]","[583.232, 736.263]","[527.34, 603.847]","[433.12, 335.976]","[542.02, 168.246]","[524.411, 288.842]","[927.599, 456.703]","[918.81, 527.314]",...,"[268.33, 527.285]","[271.191, 386.043]","[0.0, 0.0]","[0.0, 0.0]","[1380.88, 430.1]","[1560.42, 612.679]","[1545.8, 612.712]","[1492.75, 550.922]","[[165.05518248149616, 213.84598870448795, 421....",24.686542
1,"[268.25, 456.659]","[433.096, 447.823]","[436.106, 589.115]","[583.275, 736.288]","[527.359, 603.882]","[433.163, 333.091]","[539.158, 168.289]","[524.404, 286.001]","[927.644, 459.646]","[927.564, 530.267]",...,"[268.352, 527.332]","[271.216, 386.047]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[1563.31, 609.794]","[1551.62, 612.724]","[1501.59, 550.797]","[[165.08264176466284, 213.8228908980514, 421.2...",25.603550
2,"[268.286, 456.711]","[433.096, 444.981]","[436.067, 586.237]","[583.267, 736.277]","[527.377, 603.906]","[433.08, 336.003]","[542.061, 171.122]","[524.37, 286.044]","[927.645, 456.737]","[924.648, 527.362]",...,"[268.233, 527.277]","[271.322, 386.012]","[1542.86, 321.404]","[1542.81, 330.114]","[1460.36, 418.435]","[1563.3, 612.705]","[1551.57, 615.6]","[1504.48, 550.844]","[[165.22690156266927, 211.96096017191465, 421....",24.712042
3,"[268.295, 456.743]","[433.097, 444.936]","[436.05, 583.313]","[583.292, 736.209]","[527.363, 603.905]","[433.193, 333.09]","[539.132, 171.181]","[524.415, 288.862]","[927.568, 456.725]","[918.763, 527.33]",...,"[268.238, 527.274]","[271.284, 386.031]","[1545.68, 321.423]","[1545.67, 330.187]","[1451.54, 421.373]","[1563.32, 612.691]","[1551.62, 615.583]","[1501.58, 550.851]","[[165.2244063478516, 210.1468651324592, 421.09...",26.068893
4,"[268.326, 456.739]","[433.12, 444.938]","[436.038, 583.297]","[586.172, 733.39]","[527.355, 603.877]","[433.127, 336.051]","[544.94, 171.215]","[524.443, 288.888]","[930.581, 456.692]","[927.599, 527.358]",...,"[268.37, 527.302]","[271.211, 386.106]","[0.0, 0.0]","[0.0, 0.0]","[1413.28, 430.143]","[1563.36, 609.794]","[1551.58, 612.704]","[1501.54, 550.818]","[[165.21599812669473, 210.10531242212798, 421....",24.269182


## Angle between upper_arm and fore_arm_right_hand

In [73]:
n=hf.index
for i in n[:] :
    
    X = 'po_2'
    Y = 'po_3'
    Z = 'po_4'

    a = np.array(hf[X][i])
    b = np.array(hf[Y][i])
    c = np.array(hf[Z][i])
    
    ba = a - b
    bc = c - b
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    
    if np.isnan(angle):
        hf.loc[i,'hr_angle'] = 0
    else:
        hf.loc[i,'hr_angle'] = np.degrees(angle)
    


 

In [74]:
hf.head()

,po_0,po_1,po_2,po_3,po_4,po_5,po_6,po_7,po_8,po_9,...,po_18,po_19,po_20,po_21,po_22,po_23,po_24,pd,hl_angle,hr_angle
0,"[268.211, 456.696]","[433.031, 447.888]","[436.098, 589.15]","[583.232, 736.263]","[527.34, 603.847]","[433.12, 335.976]","[542.02, 168.246]","[524.411, 288.842]","[927.599, 456.703]","[918.81, 527.314]",...,"[271.191, 386.043]","[0.0, 0.0]","[0.0, 0.0]","[1380.88, 430.1]","[1560.42, 612.679]","[1545.8, 612.712]","[1492.75, 550.922]","[[165.05518248149616, 213.84598870448795, 421....",24.686542,22.119772
1,"[268.25, 456.659]","[433.096, 447.823]","[436.106, 589.115]","[583.275, 736.288]","[527.359, 603.882]","[433.163, 333.091]","[539.158, 168.289]","[524.404, 286.001]","[927.644, 459.646]","[927.564, 530.267]",...,"[271.216, 386.047]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[1563.31, 609.794]","[1551.62, 612.724]","[1501.59, 550.797]","[[165.08264176466284, 213.8228908980514, 421.2...",25.603550,22.104540
2,"[268.286, 456.711]","[433.096, 444.981]","[436.067, 586.237]","[583.267, 736.277]","[527.377, 603.906]","[433.08, 336.003]","[542.061, 171.122]","[524.37, 286.044]","[927.645, 456.737]","[924.648, 527.362]",...,"[271.322, 386.012]","[1542.86, 321.404]","[1542.81, 330.114]","[1460.36, 418.435]","[1563.3, 612.705]","[1551.57, 615.6]","[1504.48, 550.844]","[[165.22690156266927, 211.96096017191465, 421....",24.712042,21.562020
3,"[268.295, 456.743]","[433.097, 444.936]","[436.05, 583.313]","[583.292, 736.209]","[527.363, 603.905]","[433.193, 333.09]","[539.132, 171.181]","[524.415, 288.862]","[927.568, 456.725]","[918.763, 527.33]",...,"[271.284, 386.031]","[1545.68, 321.423]","[1545.67, 330.187]","[1451.54, 421.373]","[1563.32, 612.691]","[1551.62, 615.583]","[1501.58, 550.851]","[[165.2244063478516, 210.1468651324592, 421.09...",26.068893,21.005503
4,"[268.326, 456.739]","[433.12, 444.938]","[436.038, 583.297]","[586.172, 733.39]","[527.355, 603.877]","[433.127, 336.051]","[544.94, 171.215]","[524.443, 288.888]","[930.581, 456.692]","[927.599, 527.358]",...,"[271.211, 386.106]","[0.0, 0.0]","[0.0, 0.0]","[1413.28, 430.143]","[1563.36, 609.794]","[1551.58, 612.704]","[1501.54, 550.818]","[[165.21599812669473, 210.10531242212798, 421....",24.269182,20.583138


## Distance between left hand and neck

In [75]:
n=hf.index
for i in n[:] :
    
    X = 'po_1'
    Y = 'po_7'
    Z = 'po_8'

    a = np.array(hf[X][i])
    b = np.array(hf[Y][i])
    c = np.array(hf[Z][i])
    
    ab = eudis5(a,b)
    ac = eudis5(a,c)
    
    
    if ac!=0:
        dist_ratio = ab/ac
    else:
        dist_ratio = 0
    
    hf.loc[i,'hl_dist'] = dist_ratio

    

In [76]:
hf.head()

,po_0,po_1,po_2,po_3,po_4,po_5,po_6,po_7,po_8,po_9,...,po_19,po_20,po_21,po_22,po_23,po_24,pd,hl_angle,hr_angle,hl_dist
0,"[268.211, 456.696]","[433.031, 447.888]","[436.098, 589.15]","[583.232, 736.263]","[527.34, 603.847]","[433.12, 335.976]","[542.02, 168.246]","[524.411, 288.842]","[927.599, 456.703]","[918.81, 527.314]",...,"[0.0, 0.0]","[0.0, 0.0]","[1380.88, 430.1]","[1560.42, 612.679]","[1545.8, 612.712]","[1492.75, 550.922]","[[165.05518248149616, 213.84598870448795, 421....",24.686542,22.119772,0.370827
1,"[268.25, 456.659]","[433.096, 447.823]","[436.106, 589.115]","[583.275, 736.288]","[527.359, 603.882]","[433.163, 333.091]","[539.158, 168.289]","[524.404, 286.001]","[927.644, 459.646]","[927.564, 530.267]",...,"[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[1563.31, 609.794]","[1551.62, 612.724]","[1501.59, 550.797]","[[165.08264176466284, 213.8228908980514, 421.2...",25.603550,22.104540,0.375599
2,"[268.286, 456.711]","[433.096, 444.981]","[436.067, 586.237]","[583.267, 736.277]","[527.377, 603.906]","[433.08, 336.003]","[542.061, 171.122]","[524.37, 286.044]","[927.645, 456.737]","[924.648, 527.362]",...,"[1542.86, 321.404]","[1542.81, 330.114]","[1460.36, 418.435]","[1563.3, 612.705]","[1551.57, 615.6]","[1504.48, 550.844]","[[165.22690156266927, 211.96096017191465, 421....",24.712042,21.562020,0.370498
3,"[268.295, 456.743]","[433.097, 444.936]","[436.05, 583.313]","[583.292, 736.209]","[527.363, 603.905]","[433.193, 333.09]","[539.132, 171.181]","[524.415, 288.862]","[927.568, 456.725]","[918.763, 527.33]",...,"[1545.68, 321.423]","[1545.67, 330.187]","[1451.54, 421.373]","[1563.32, 612.691]","[1551.62, 615.583]","[1501.58, 550.851]","[[165.2244063478516, 210.1468651324592, 421.09...",26.068893,21.005503,0.365592
4,"[268.326, 456.739]","[433.12, 444.938]","[436.038, 583.297]","[586.172, 733.39]","[527.355, 603.877]","[433.127, 336.051]","[544.94, 171.215]","[524.443, 288.888]","[930.581, 456.692]","[927.599, 527.358]",...,"[0.0, 0.0]","[0.0, 0.0]","[1413.28, 430.143]","[1563.36, 609.794]","[1551.58, 612.704]","[1501.54, 550.818]","[[165.21599812669473, 210.10531242212798, 421....",24.269182,20.583138,0.363360


## Distance between right hand and neck

In [77]:
n=hf.index
for i in n[:] :
    
    X = 'po_1'
    Y = 'po_4'
    Z = 'po_8'

    a = np.array(hf[X][i])
    b = np.array(hf[Y][i])
    c = np.array(hf[Z][i])
    
    ab = eudis5(a,b)
    ac = eudis5(a,c)
    
    if ac!=0:
        dist_ratio = ab/ac
    else:
        dist_ratio = 0
    
    hf.loc[i,'hr_dist'] = dist_ratio

    

In [78]:
hf.head()

,po_0,po_1,po_2,po_3,po_4,po_5,po_6,po_7,po_8,po_9,...,po_20,po_21,po_22,po_23,po_24,pd,hl_angle,hr_angle,hl_dist,hr_dist
0,"[268.211, 456.696]","[433.031, 447.888]","[436.098, 589.15]","[583.232, 736.263]","[527.34, 603.847]","[433.12, 335.976]","[542.02, 168.246]","[524.411, 288.842]","[927.599, 456.703]","[918.81, 527.314]",...,"[0.0, 0.0]","[1380.88, 430.1]","[1560.42, 612.679]","[1545.8, 612.712]","[1492.75, 550.922]","[[165.05518248149616, 213.84598870448795, 421....",24.686542,22.119772,0.370827,0.368458
1,"[268.25, 456.659]","[433.096, 447.823]","[436.106, 589.115]","[583.275, 736.288]","[527.359, 603.882]","[433.163, 333.091]","[539.158, 168.289]","[524.404, 286.001]","[927.644, 459.646]","[927.564, 530.267]",...,"[0.0, 0.0]","[0.0, 0.0]","[1563.31, 609.794]","[1551.62, 612.724]","[1501.59, 550.797]","[[165.08264176466284, 213.8228908980514, 421.2...",25.603550,22.104540,0.375599,0.368551
2,"[268.286, 456.711]","[433.096, 444.981]","[436.067, 586.237]","[583.267, 736.277]","[527.377, 603.906]","[433.08, 336.003]","[542.061, 171.122]","[524.37, 286.044]","[927.645, 456.737]","[924.648, 527.362]",...,"[1542.81, 330.114]","[1460.36, 418.435]","[1563.3, 612.705]","[1551.57, 615.6]","[1504.48, 550.844]","[[165.22690156266927, 211.96096017191465, 421....",24.712042,21.562020,0.370498,0.373541
3,"[268.295, 456.743]","[433.097, 444.936]","[436.05, 583.313]","[583.292, 736.209]","[527.363, 603.905]","[433.193, 333.09]","[539.132, 171.181]","[524.415, 288.862]","[927.568, 456.725]","[918.763, 527.33]",...,"[1545.67, 330.187]","[1451.54, 421.373]","[1563.32, 612.691]","[1551.62, 615.583]","[1501.58, 550.851]","[[165.2244063478516, 210.1468651324592, 421.09...",26.068893,21.005503,0.365592,0.373660
4,"[268.326, 456.739]","[433.12, 444.938]","[436.038, 583.297]","[586.172, 733.39]","[527.355, 603.877]","[433.127, 336.051]","[544.94, 171.215]","[524.443, 288.888]","[930.581, 456.692]","[927.599, 527.358]",...,"[0.0, 0.0]","[1413.28, 430.143]","[1563.36, 609.794]","[1551.58, 612.704]","[1501.54, 550.818]","[[165.21599812669473, 210.10531242212798, 421....",24.269182,20.583138,0.363360,0.371333


## Filtering the dataframe with desired columns

In [79]:
df= pd.DataFrame()

In [80]:
#df=hf.filter(items=['pd', 'hl_angle', 'hr_angle', 'hl_dist', 'hr_dist'])
df=hf.filter(items=['hl_angle', 'hr_angle', 'hl_dist', 'hr_dist'])

In [81]:
df.head(1)

,hl_angle,hr_angle,hl_dist,hr_dist
0,24.686542,22.119772,0.370827,0.368458


In [82]:
#df = df.join(df['pd'].apply(pd.Series).add_prefix('p_'))

In [83]:
df.head(1)

,hl_angle,hr_angle,hl_dist,hr_dist
0,24.686542,22.119772,0.370827,0.368458


In [84]:
#df = df.join(df['p_0'].apply(pd.Series).add_prefix('pd_'))

In [85]:
df.head(1)

,hl_angle,hr_angle,hl_dist,hr_dist
0,24.686542,22.119772,0.370827,0.368458


In [86]:
#df = df.drop(['pd', 'p_0'], axis=1)

In [87]:
df.head(1)

,hl_angle,hr_angle,hl_dist,hr_dist
0,24.686542,22.119772,0.370827,0.368458


In [88]:
df.drop([col for col, val in df.sum().iteritems() if val == 0], axis=1, inplace=True)

In [89]:
df.head(1)

,hl_angle,hr_angle,hl_dist,hr_dist
0,24.686542,22.119772,0.370827,0.368458


In [90]:
df.to_csv('/mnt/sde/jagadish/userdata/dl_project/hc_test_data/hc_8379.csv', index=True)

## Code below is to drop certain negative samples in training data

In [ ]:
test = pd.read_table("/mnt/sde/jagadish/userdata/dl_project/tv_train_files/tv_8292.csv",sep=",")

In [ ]:
df = test

In [ ]:
df.head(1)

In [ ]:

n = df.index
for i in n[6:-6] :
    df.loc[i,'W'] = df['Y'].iloc[i] + df['Y'].iloc[i-6] + df['Y'].iloc[i-5] + df['Y'].iloc[i-4] + df['Y'].iloc[i-3] + df['Y'].iloc[i-2] + df['Y'].iloc[i-1] + df['Y'].iloc[i+1] + df['Y'].iloc[i+2] + df['Y'].iloc[i+3] + df['Y'].iloc[i+4] + df['Y'].iloc[i+5] + df['Y'].iloc[i+6]   
        


                                    


In [ ]:
df.head(10)

In [ ]:
df=df.drop(df[(df['Y'] == 0) & (df['W'] != 0)].index)

In [ ]:
df.head(1)

In [ ]:
df.to_csv('/mnt/sde/jagadish/userdata/dl_project/tv_new_train_files/tv_8345_new.csv', index=True)